## Este código es una prueba de un agente más inteligente que recibe la lista de celdas sucias en orden. No es el Notebook para la entrega calificada, sino una referencia personal.

In [ ]:
!pip install agentpy
!pip install seaborn

In [2]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import HTML
import agentpy as ap

In [166]:
def my_plot(model, ax):
    # Create a 3D grid for RGB values
    grid = np.ones((*model.environment.shape, 3))  # initialize as white

    for agent, pos in model.environment.positions.items():
        if agent.__class__.__name__ == "CleanerAgent":
            # Agent1 is always black
            grid[pos] = [0, 0, 0]  
        elif agent.__class__.__name__ == "FloorCell":
            if agent.cleaning:
                grid[pos] = [0, 0, 0] # black
            elif agent.clean:
                grid[pos] = [1, 1, 1] #white
            else:
                grid[pos] = [0.59, 0.29, 0] #brown
    
    ax.imshow(grid)

In [180]:
class CleanerAgent(ap.Agent):

    # Setting move direction
    def setup(self, env):
        self.direction = (0,0)
        self.env = env
        self.dirtyCellsList = []
        self.dirtyCellIdx = 0

    # Gets the agent's position directly from environment
    def get_position(self):
        return self.env.positions[self]

    def set_dirty(self,dirtyCells):
        self.dirtyCellsList = dirtyCells
    
    
    def move_to_cell(self,dirtyCell, position):
        direction = (0,0)

        if(dirtyCell[0] == position[0]):
            if(dirtyCell[1] < position[1]):
                direction = (0,-1)
            else:
                direction = (0,1)
        else:
            if(dirtyCell[0] < position[0]):
                direction = (-1,0)
            else:
                direction = (1,0)
        return direction

    def execute(self):
        # Get position of cell of next movement.
        if(self.dirtyCellIdx < len(self.dirtyCellsList)):
            nextDirtyCell = self.dirtyCellsList[self.dirtyCellIdx]

            if(self.get_position() == nextDirtyCell):
                #In Dirty Cell Clean and leave
                self.dirtyCellIdx += 1
                if(self.dirtyCellIdx >= len(self.dirtyCellsList)):
                    return
                nextDirtyCell = self.dirtyCellsList[self.dirtyCellIdx]
                    
            self.direction = self.move_to_cell(nextDirtyCell, self.get_position())
            self.env.move_by(self, self.direction)
        
            


In [186]:
class FloorCell(ap.Agent):
    def setup(self, env, loc, cleaner, clean):
        self.env = env
        self.location = loc
        self.cleaner = cleaner
        self.clean = clean
        self.cleaning = False
        if(self.location == self.env.positions[self.cleaner]):
            self.cleaning = True


    def execute(self):
        if(self.location == self.env.positions[self.cleaner]):
            self.cleaning = True
        elif (self.cleaning):
            if(self.p.print):
                print("Cleaning Cell at {}".format(self.location))
            self.cleaning = False
            self.clean = True


In [242]:
class CleanerModel(ap.Model):

    def setup(self):
        # System = ((Ag1, Ag2), Env, ())
        # The agents in this system will only move left or right, but should not collide!!!
        m = self.p.m
        n = self.p.n
        self.environment = ap.Grid(self, (m, n))
        self.dirtyCells = []
        self.cleanerAgent = CleanerAgent(self, self.environment)
        self.environment.add_agents([self.cleanerAgent], positions=[(0,0)])
        self.dirtyCellsAgents = []
        cellPositions = []
        switch = False
        for i in range(n):
            if(not switch):
                rowRange = range(0,m,1)
            else:
                rowRange = range(m-1,-1,-1)
            
            switch = not switch
            for j in rowRange:
                clean = np.random.choice([True,False])
                self.dirtyCellsAgents.append(FloorCell(self, self.environment, (j,i), self.cleanerAgent,clean))
                if(not clean):
                    self.dirtyCells.append((j,i))
                cellPositions.append((j,i))
        self.cleanerAgent.set_dirty(self.dirtyCells)
        self.environment.add_agents(self.dirtyCellsAgents, positions=cellPositions)
        if(self.p.print):
            print("Cleaner agent position:{}".format(self.environment.positions[self.cleanerAgent]))

    def step(self):
        
        self.environment.agents.execute()
        if(self.p.print):
            print("\n********************")
            print("Cleaner agent position:{}".format(self.environment.positions[self.cleanerAgent]))

In [246]:
fig, ax = plt.subplots()
parameters = {'print': True, 'steps':90, 'm':12, 'n':8} 
cleanerModel = CleanerModel(parameters)
animation = ap.animate(cleanerModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

Cleaner agent position:(0, 0)
Cleaning Cell at (0, 0)

********************
Cleaner agent position:(1, 0)
Cleaning Cell at (1, 0)

********************
Cleaner agent position:(2, 0)
Cleaning Cell at (2, 0)

********************
Cleaner agent position:(3, 0)
Cleaning Cell at (3, 0)

********************
Cleaner agent position:(4, 0)
Cleaning Cell at (4, 0)

********************
Cleaner agent position:(5, 0)
Cleaning Cell at (5, 0)

********************
Cleaner agent position:(6, 0)
Cleaning Cell at (6, 0)

********************
Cleaner agent position:(7, 0)
Cleaning Cell at (7, 0)

********************
Cleaner agent position:(8, 0)
Cleaning Cell at (8, 0)

********************
Cleaner agent position:(9, 0)
Cleaning Cell at (9, 0)

********************
Cleaner agent position:(10, 0)
Cleaning Cell at (10, 0)

********************
Cleaner agent position:(11, 0)
Cleaning Cell at (11, 0)

********************
Cleaner agent position:(11, 1)
Cleaning Cell at (11, 1)

********************
Cleane

# Uso de Chat GPT

## Prompts
"
```python
def my_plot(model, ax):
    grid = np.zeros(model.environment.shape)
    print(model.environment.positions)
    for agent, pos in model.environment.positions.items():
        grid[pos] = agent.id
    ax.imshow(grid, cmap='Greys')
```
I have two types of agents.
The agent of class Agent1 never changes color and is black.
The agent of class Agent2 has an attribute called clean.
If Agent2 is clean, make its color white, if it is not clean, make its color brown.
"

## Cambios
Modifiqué la respuesta de ChatGPT porque no había considerado la lógica de cuando el agente de limpieza estuviera "encima" de una celda.
En ese caso, no se mostraba el color negro del agente.